# ML Assignment 6 - Sample Code
* 雲端硬碟: https://drive.google.com/drive/folders/1KqXE_drqYYwg9RsQil3oXQeskXzuATdR?usp=sharing
* 蘭花競賽網站: https://tbrain.trendmicro.com.tw/Competitions/Details/20

## 執行方式
資料集請去雲端硬碟取得(./dataset)，內有壓縮檔可下載

依作業要求，在圖像轉換區塊更改程式碼。
訓練過程及輸出位於最後面。


## 初始設定

In [ ]:
import time
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.hub import load_state_dict_from_url
import torchvision
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import ConcatDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd


## 圖像轉換
### 題目
torchvision.transforms 提供了許多可靠的 API來讓使用者對圖像進行操作，請試著在 data transforms 當中對訓練集進行轉換(圖像前處理)，當模型訓練到一定程度時，驗證看看使用該方法是否確實對模型準確率造成影響

* **Weak Augmentation** - 使用**1**種data transforms，並記錄其**使用前、使用後的validation accuracy**，共做1~3次

* **Strong Augmentation** - 使用**4~6**種data transforms，並記錄其**使用前、使用後的validation accuracy**

### 下面列出目前全部可用的transforms，參數部分自行Google :)



In [ ]:
class MyCNN(nn.Module):

  def __init__(self, num_classes=1000):
    super(MyCNN, self).__init__()
    self.features = nn.Sequential(
      nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2),
      nn.Conv2d(64, 192, kernel_size=5, padding=2),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2),
      nn.Conv2d(192, 384, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.Conv2d(384, 256, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.Conv2d(256, 256, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
    self.classifier = nn.Sequential(
      nn.Dropout(),
      nn.Linear(256 * 6 * 6, 4096),
      nn.ReLU(inplace=True),
      nn.Dropout(),
      nn.Linear(4096, 4096),
      nn.ReLU(inplace=True),
    )
    self.classifier2 = nn.Sequential(
      nn.Linear(4096, num_classes),
    )

  def forward(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    x = self.classifier2(x)

    return x

## 訓練模型區塊
包含視覺化模型及訓練模型。

In [ ]:
def visualize_model(
    model, device, dataloaders, class_names, num_images=6,
    savepath=os.path.join("default")
  ):
  if not os.path.exists(savepath):
    os.mkdir(savepath)
  
  was_training = model.training
  model.eval()
  images_so_far = 0

  plt.figure(figsize=(18,9))

  with torch.no_grad():
    for i, (inputs, labels) in enumerate(dataloaders['val']):
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)

      for j in range(inputs.size()[0]):
        images_so_far += 1

        img_display = np.transpose(inputs.cpu().data[j].numpy(), (1,2,0)) #numpy:CHW, PIL:HWC
        plt.subplot(num_images//2,2,images_so_far),plt.imshow(img_display) #nrow,ncol,image_idx
        plt.title(f'predicted: {class_names[preds[j]]}')
        plt.savefig(os.path.join(savepath,"test.jpg"))
        if images_so_far == num_images:
            model.train(mode=was_training)
            plt.clf()
            return
    plt.clf()
    model.train(mode=was_training)

In [ ]:
def imshow(inp, title=None):
  """Imshow for Tensor."""
  inp = inp.numpy().transpose((1, 2, 0))
  mean = np.array([0.485, 0.456, 0.406])
  std = np.array([0.229, 0.224, 0.225])
  
  #原先Normalize是對每個channel個別做 減去mean, 再除上std
  inp1 = std * inp + mean

  plt.imshow(inp)

  if title is not None:
      plt.title(title)
  plt.pause(0.001)  # pause a bit so that plots are updated
  plt.imshow(inp1)
  if title is not None:
      plt.title(title)
  plt.clf()

In [ ]:

def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def train_model(
    model, criterion, device, 
    dataloaders, dataset_sizes, optimizer, scheduler, 
    num_epochs=25
):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  train_loss, valid_loss = [], []
  train_acc, valid_acc = [], []

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()  # Set model to training mode
      else:
        model.eval()   # Set model to evaluate mode

      running_loss = 0.0
      running_corrects = 0

      # Iterate over data.
      for inputs, labels in tqdm(dataloaders[phase]):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # backward + optimize only if in training phase
            if phase == 'train':
                # zero the parameter gradients
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]

      if phase == 'train':
        train_loss.append(epoch_loss)
        train_acc.append(epoch_acc.cpu().item())
      else:
        valid_loss.append(epoch_loss)
        valid_acc.append(epoch_acc.cpu().item())

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        phase, epoch_loss, epoch_acc))

      # deep copy the model
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())


  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  # load best model weights
  model.load_state_dict(best_model_wts)
  #torch.save(model.state_dict(),"model.pt")
  return model, {
    'train loss':train_loss,
    'train acc':train_acc,
    'val loss':valid_loss,
    'val acc':valid_acc
  }

In [ ]:
def plot_hist_cmp(train_loss:dict, valid_loss:dict, root=os.path.join("cmp")):
    
    if not os.path.exists(root):
        os.mkdir(root)
    
    plt.figure(dpi=800)
    for name, tloss in train_loss.items():
        plt.plot(range(1,len(tloss)+1,1), np.array(tloss), label= name) 
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.legend()
    plt.savefig(os.path.join(root,"cmp_train_loss.jpg"))
    plt.close()

    plt.figure(dpi=800)
    for name, vloss in valid_loss.items():
        plt.plot(range(1,len(vloss)+1,1), np.array(vloss), label= name) #--evaluate_during_training True 在啟用eval
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.legend()
    plt.savefig(os.path.join(root, "cmp_eval_loss.jpg"))
    plt.close()

## 主函式
- hyp:
    - num_epochs: 訓練回合數
    - lr: 訓練速度(learning rate)
    - batch_size: 批次(batch)大小
- preprocess:
    圖片要放進 model 前要做的處理 
    - (不用放 ```transforms.ToTensor()```)
- argumentation:
    argumentation methods:
    - a 2D list, each one contains 1 argumentation method

In [ ]:
def main(data_dir :os.PathLike, preprocess:list, 
    argumentation:list = None, 
    hyp = {'num_epochs':20, 'lr':0.001,'batchsize': 64},
    vismodel_savepath = os.path.join("default") ):

    num_workers = 2
    momentum = 0.9
    
    train_data_dir = os.path.join(data_dir, "train")
    valid_data_dir = os.path.join(data_dir, "val")
    
    to_tensor = [transforms.ToTensor()] 
    default_preprocess = transforms.Compose(preprocess + to_tensor)
    train_prepreocess = ImageFolder(train_data_dir, default_preprocess)
    class_names = train_prepreocess.classes
    train_img_datasets = [train_prepreocess]
    bsize = hyp["batchsize"]
    if argumentation is not None:
        for arg_method in argumentation:
           
            t = arg_method + preprocess + to_tensor
            print(t)
            argumentation_trans = transforms.Compose(t)
            train_img_datasets.append(
                ImageFolder(train_data_dir,argumentation_trans)
            )
    train_img_datasets = ConcatDataset(train_img_datasets)
    
    valid_img_datasets = ImageFolder(
        valid_data_dir,default_preprocess
    )
    
    dataloaders = {
        'train': DataLoader(
            train_img_datasets, batch_size=bsize,
            shuffle=True, num_workers=num_workers
        ) ,
        'val': DataLoader(
            valid_img_datasets,batch_size=bsize,
            shuffle=True, num_workers=num_workers
        )
    } 
    dataset_sizes = {
        'train': len(train_img_datasets),
        'val': len(valid_img_datasets)
    }
    print(dataset_sizes)
    
  
    device = torch.device(
        "cuda:4" if torch.cuda.is_available() else "cpu"
    )
    print(f"Using device {device}\n")

    # Get a batch of training data
    inputs, classes = next(iter(dataloaders['train']))
    # Make a grid from batch
    out = torchvision.utils.make_grid(inputs)
    
    
    imshow(out, title=[class_names[x] for x in classes])


    model_ft = MyCNN(num_classes=219)
    pretrained_dict = load_state_dict_from_url(
        'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
        progress=True
    )
    model_dict = model_ft.state_dict()
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict) 
    # 3. load the new state dict
    model_ft.load_state_dict(model_dict)

    #for k,v in model_dict.items():
    #  print(k)

    model_ft = model_ft.to(device)


    parameter_count = count_parameters(model_ft)
    print(f"#parameters:{parameter_count}")
    print(f"batch_size:{bsize}")


    criterion = nn.CrossEntropyLoss()
    optimizer_ft = optim.SGD(
        model_ft.parameters(), lr=hyp['lr'], momentum=momentum
    )
    exp_lr_scheduler = lr_scheduler.StepLR(
        optimizer_ft, step_size=7, gamma=0.1
    )
    model_ft, history = train_model(
        model_ft, criterion, device, 
        dataloaders, dataset_sizes, optimizer_ft, 
        exp_lr_scheduler, num_epochs=hyp['num_epochs']
    )
    class_names = valid_img_datasets.classes
    visualize_model(
        model_ft, device, 
        dataloaders, class_names, 
        savepath=vismodel_savepath
    )
    return history


In [ ]:
hyp = {
        'num_epochs':20, 'lr':0.001,
        'batchsize': 64
}

In [ ]:
data_preprocessing = [
    transforms.Resize((224,224) )
]


## Defualt

In [ ]:
default_metrics = main(
    data_dir=os.path.join("training"),
    preprocess=data_preprocessing, hyp=hyp,
    vismodel_savepath=os.path.join("default")
)

torch.cuda.empty_cache()

## Weak argumentation

with 1 method => len( dataset ) = 2*len(origin dataset ) 

In [ ]:
argumentation =[ 
    [
        transforms.GaussianBlur(kernel_size=21, sigma=5)
    ]
]

In [ ]:
guassian_metrics = main(
    data_dir=os.path.join("training"),
    preprocess=data_preprocessing,
    hyp=hyp,
    argumentation=argumentation,
    vismodel_savepath=os.path.join("guassianblur")
)

torch.cuda.empty_cache()

## Strong argumentation

with 1 method => len( dataset ) = 2*len(origin dataset ) 

In [ ]:

#strong 
s_argumentation = [
    [
        transforms.GaussianBlur(kernel_size=21, sigma=5),
        transforms.RandomAffine(degrees=(-30,30)),
        transforms.ColorJitter(brightness=(0, 5)),
        transforms.CenterCrop(size=300)
    
    ]
]

strong_arg_metrics =  main(
    data_dir=os.path.join("training"),
    preprocess=data_preprocessing,
    hyp=hyp,
    argumentation=s_argumentation,
    vismodel_savepath=os.path.join("strong")
)


## CMP

In [ ]:
trains = {
    'default':default_metrics['train loss'],
    '+gussianblur':guassian_metrics['train loss'],
    'strong':strong_arg_metrics['train loss']
}

valids = {
    'default':default_metrics['val loss'],
    '+gussianblur' : guassian_metrics['val loss'],
    'strong':strong_arg_metrics['val loss']
}
r = os.path.join("cmp")
if not os.path.exists(r):
    os.mkdir(r)

plot_hist_cmp(
    train_loss=trains, valid_loss=valids,root=r
)

In [ ]:
best_val_acc = pd.DataFrame(
    {
        'default':[max(default_metrics['val acc'])],
        '+gussianblur':[max(guassian_metrics['val acc'])],
        'strong':[max(strong_arg_metrics['val acc'])]
    }
)
best_val_acc.to_csv(
    os.path.join(r,"best_val_acc.csv"), 
    index=False
)
